In [11]:
import pyodbc 
import pandas as pd
import datetime
import numpy as np

In [4]:
'''Connect to post result SQL server'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=dsa-ln-TQA1V;DATABASE=qai;Trusted_Connection=yes')   
cursor = cnxn.cursor() 

In [5]:
datetime.datetime.now()

datetime.datetime(2018, 12, 21, 10, 54, 24, 538970)

In [6]:
'''
Broker level estimates
'''

ibdestl_sql="select * from IBDEstL1 where PerType=1 and Period<3 and year(EstDate)>2016 and year(EstDate)<2018"
ibdestl=pd.read_sql(ibdestl_sql,cnxn)

In [61]:
'''
Dataframe with mean, median, count
mean estimate vs previous consensus
mean revision vs previous consensus
mean revision vs previous estimate
same thing applies to year 1 and 2
'''

In [195]:
def key_IBES_metrics(code,date,est_period,revision_period,broker_df,consensus_df):
    '''
    utility function to generate key metrics from broker level data and consensus data
    '''
    broker1=broker_df[(broker_df.Code==code)&(broker_df.Period==1)]
    broker2=broker_df[(broker_df.Code==code)&(broker_df.Period==2)]
    
    consensus1=consensus_df[(consensus_df.Code==code)&(consensus_df.Period==1)]
    consensus2=consensus_df[(consensus_df.Code==code)&(consensus_df.Period==2)]                 
    
    broker1_est=broker1[(broker1.EstDate>=date)&(broker1.EstDate<=date+datetime.timedelta(est_period))]
    broker2_est=broker2[(broker2.EstDate>=date)&(broker2.EstDate<=date+datetime.timedelta(est_period))]
    
    pre_consensus1=consensus1[(consensus1.Code==code)&(consensus1.EstDate<=date)].sort_values("EstDate").iloc[-1]["Mean"]
    post_consensus1=consensus1[(consensus1.Code==code)&(consensus1.EstDate>=date+datetime.timedelta(revision_period))].\
    sort_values("EstDate").iloc[0]["Mean"]
    
    pre_consensus2=consensus2[(consensus2.Code==code)&(consensus2.EstDate<=date)].sort_values("EstDate").iloc[-1]["Mean"]
    post_consensus2=consensus2[(consensus2.Code==code)&(consensus2.EstDate>=date+datetime.timedelta(revision_period))].\
    sort_values("EstDate").iloc[0]["Mean"]
    
    broker1_mean_from_consensus=broker1_est["Value_"].mean()/pre_consensus1-1
    broker1_mean_revision=(broker1_est["Value_"]/pre_consensus1-1).mean()
    broker2_mean_from_consensus=broker2_est["Value_"].mean()/pre_consensus2-1
    broker2_mean_revision=(broker2_est["Value_"]/pre_consensus2-1).mean()

    broker1_median_from_consensus=broker1_est["Value_"].mean()/pre_consensus1-1
    broker1_median_revision=(broker1_est["Value_"]/pre_consensus1-1).median()
    broker2_median_from_consensus=broker2_est["Value_"].mean()/pre_consensus2-1
    broker2_median_revision=(broker2_est["Value_"]/pre_consensus2-1).median()
    
    actual_revision1=post_consensus1/pre_consensus1-1
    actual_revision2=post_consensus2/pre_consensus2-1
    
    count1=broker1_est.shape[0]
    count2=broker2_est.shape[0]
    
    print(pre_consensus1,post_consensus1,broker1_est)
    return broker1_mean_from_consensus,broker1_mean_revision,broker2_mean_from_consensus,broker2_mean_revision,\
           broker1_median_from_consensus,broker1_median_revision,broker2_median_from_consensus,broker2_median_revision,\
           actual_revision1,actual_revision2,count1,count2

    

In [235]:
target=alist[609]

In [236]:
target

(1272, Timestamp('2017-02-08 00:00:00'), Timestamp('2016-12-31 00:00:00'))

In [237]:
key_IBES_metrics(target[0],target[1],2,20,ibdestl,ibesusdlyest)

1.13199996948242 1.16250002384186        Code  Measure  Broker  Analyst  PerType  Period    EstDate    PerDate  \
20304  1272        8      31    32632        1       1 2017-02-08 2017-12-31   
20321  1272        8   50155   506136        1       1 2017-02-08 2017-12-31   
20502  1272        8   51534   614208        1       1 2017-02-09 2017-12-31   
20515  1272        8   52246   524702        1       1 2017-02-09 2017-12-31   

      Currency_ PDFlag  Value_    RevDate IsoCurrency  
20304                D    1.20 2017-07-12        None  
20321                D    1.04 2017-04-04        None  
20502                D    1.26 2017-02-19        None  
20515                D    1.24 2017-02-09        None  


(0.04681981620707365,
 0.04681981620707365,
 0.02434454001824693,
 0.024344540018246874,
 0.04681981620707365,
 0.07773854495580579,
 0.02434454001824693,
 0.02247188089572727,
 0.02694351164460307,
 0.041198461405485665,
 4,
 4)

In [238]:
target

(1272, Timestamp('2017-02-08 00:00:00'), Timestamp('2016-12-31 00:00:00'))

In [239]:
ibdestl[(ibdestl.Code==target[0])&(ibdestl.EstDate>=target[1])&(ibdestl.EstDate<=target[1]+datetime.timedelta(2))]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency
20304,1272,8,31,32632,1,1,2017-02-08,2017-12-31,,D,1.20,2017-07-12,None
20308,1272,8,31,32632,1,2,2017-02-08,2018-12-31,,D,1.35,2017-07-12,None
20321,1272,8,50155,506136,1,1,2017-02-08,2017-12-31,,D,1.04,2017-04-04,None
20327,1272,8,50155,506136,1,2,2017-02-08,2018-12-31,,D,1.25,2017-04-04,None
20502,1272,8,51534,614208,1,1,2017-02-09,2017-12-31,,D,1.26,2017-02-19,None
20509,1272,8,51534,614208,1,2,2017-02-09,2018-12-31,,D,1.49,2017-02-19,None
20515,1272,8,52246,524702,1,1,2017-02-09,2017-12-31,,D,1.24,2017-02-09,None
20522,1272,8,52246,524702,1,2,2017-02-09,2018-12-31,,D,1.38,2017-02-09,None


In [240]:
ibdestl[(ibdestl.Code==target[0])]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency
20304,1272,8,31,32632,1,1,2017-02-08,2017-12-31,,D,1.200,2017-07-12,None
20305,1272,8,31,32632,1,1,2017-07-20,2017-12-31,,D,1.260,2017-09-25,None
20306,1272,8,31,32632,1,1,2017-10-13,2017-12-31,,D,1.240,2017-10-16,None
20307,1272,8,31,32632,1,1,2017-10-26,2017-12-31,,D,1.252,2017-10-26,None
20308,1272,8,31,32632,1,2,2017-02-08,2018-12-31,,D,1.350,2017-07-12,None
20309,1272,8,31,32632,1,2,2017-07-20,2018-12-31,,D,1.450,2017-09-25,None
20310,1272,8,31,32632,1,2,2017-10-13,2018-12-31,,D,1.420,2017-10-13,None
20311,1272,8,31,32632,1,2,2017-10-26,2018-12-31,,D,1.451,2017-10-26,None
20312,1272,8,3322,143179,1,1,2017-04-19,2017-12-31,,D,1.170,2017-04-19,None
20313,1272,8,3322,143179,1,1,2017-04-26,2017-12-31,,D,1.200,2017-04-26,None


In [180]:
'''
Pre value
'''
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate<=target[1])&(ibesusdlyest.Period==1)].\
sort_values("EstDate").iloc[-1]["Mean"]

6.44168758392334

In [181]:
'''
Post value
'''
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate>=target[1]+datetime.timedelta(20))&(ibesusdlyest.Period==1)].\
sort_values("EstDate").iloc[0]["Mean"]

6.38970565795898

In [187]:
ibesusdlyest[ibesusdlyest.Code==target[0]]

,Code,Measure,EstDate,PerDate,Period,PerType,NumEst,Median,Mean,StdDev,High,Low,PdFlag,Currency_
1947,330,8,2017-02-17,2018-12-31,2,1,17,6.7000,6.706000,0.087175,6.850,6.540,D,USD
2010,330,8,2017-04-18,2017-12-31,1,1,17,6.4400,6.426764,0.058495,6.510,6.320,D,USD
2011,330,8,2017-04-20,2017-12-31,1,1,17,6.4450,6.436765,0.051019,6.510,6.320,D,USD
2012,330,8,2017-04-24,2017-12-31,1,1,17,6.4500,6.438824,0.052068,6.510,6.320,D,USD
2023,330,8,2017-04-03,2017-12-31,1,1,17,6.4000,6.406177,0.060610,6.510,6.320,D,USD
2024,330,8,2017-04-06,2017-12-31,1,1,17,6.4000,6.412059,0.064688,6.510,6.320,D,USD
2025,330,8,2017-04-13,2017-12-31,1,1,17,6.4000,6.417941,0.063222,6.510,6.320,D,USD
2026,330,8,2017-07-25,2017-12-31,1,1,16,6.5050,6.506875,0.054002,6.600,6.400,D,USD
2027,330,8,2017-04-03,2018-12-31,2,1,17,6.7000,6.716000,0.092382,6.850,6.540,D,USD
2028,330,8,2017-04-06,2018-12-31,2,1,17,6.7000,6.721882,0.103912,6.950,6.540,D,USD


In [194]:
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate>=target[1]+datetime.timedelta(20))&(ibesusdlyest.Period==1)].\
sort_values("EstDate")

,Code,Measure,EstDate,PerDate,Period,PerType,NumEst,Median,Mean,StdDev,High,Low,PdFlag,Currency_
7436,919,8,2017-02-28,2017-12-31,1,1,10,6.980,7.001000,0.125738,7.25,6.840,D,USD
7440,919,8,2017-03-01,2017-12-31,1,1,10,6.980,7.026000,0.189455,7.50,6.840,D,USD
7706,919,8,2017-03-21,2017-12-31,1,1,10,6.980,7.029000,0.189763,7.50,6.840,D,USD
7707,919,8,2017-03-24,2017-12-31,1,1,10,7.030,7.040000,0.187202,7.50,6.840,D,USD
7708,919,8,2017-04-10,2017-12-31,1,1,10,7.030,7.035000,0.173734,7.45,6.840,D,USD
7638,919,8,2017-05-04,2017-12-31,1,1,10,7.065,7.080000,0.184451,7.45,6.840,D,USD
7565,919,8,2017-05-05,2017-12-31,1,1,10,7.115,7.108000,0.181647,7.40,6.700,D,USD
7324,919,8,2017-06-23,2017-12-31,1,1,10,7.115,7.121000,0.151324,7.40,6.830,D,USD
7787,919,8,2017-07-18,2017-12-31,1,1,10,7.115,7.125000,0.154434,7.40,6.830,D,USD
7788,919,8,2017-07-19,2017-12-31,1,1,10,7.140,7.138000,0.162056,7.40,6.830,D,USD


In [7]:
'''
Daily consensus data
'''

ibesusdlyest_sql=\
"select * from IBESUSDLYEST where PerType=1 and Period<3 and Measure=8 and year(EstDate)>2016 and year(EstDate)<2018"
ibesusdlyest=pd.read_sql(ibesusdlyest_sql,cnxn)

In [37]:
'''
Actual reporting data
'''

ibdactl_sql="select * from IBDActL1 where PerType='Q' and year(RptDate)>2016 and year(RptDate)<2018"
ibdactl=pd.read_sql(ibdactl_sql,cnxn)

In [88]:
len(alist)

21264

In [94]:
alist=list(zip(ibdactl.Code,ibdactl.RptDate,ibdactl.PerDate))

In [191]:
target=alist[500]

In [193]:
ibdestl[(ibdestl.Code==target[0])&(ibdestl.EstDate>=target[1])&(ibdestl.EstDate<=target[1]+datetime.timedelta(2))]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency
15033,919,8,228,187538,1,2,2017-02-07,2018-12-31,,D,7.350,2017-05-04,None
15039,919,8,238,117325,1,1,2017-02-08,2017-12-31,,D,6.950,2017-02-08,None
15043,919,8,238,117325,1,2,2017-02-08,2018-12-31,,D,7.550,2017-02-08,None
15053,919,8,260,76873,1,2,2017-02-07,2018-12-31,,D,7.490,2017-03-19,None
15062,919,8,2301,83036,1,1,2017-02-08,2017-12-31,,D,6.940,2017-02-15,None
15799,919,8,2301,83036,1,2,2017-02-08,2018-12-31,,D,7.410,2017-02-15,None
15807,919,8,3322,185662,1,1,2017-02-07,2017-12-31,,D,6.890,2017-02-07,None
15811,919,8,3322,185662,1,2,2017-02-07,2018-12-31,,D,7.200,2017-02-07,None
15817,919,8,50203,619739,1,2,2017-02-08,2018-12-31,,D,7.750,2017-05-04,None
15821,919,8,50531,640084,1,1,2017-02-07,2017-12-31,,D,6.995,2017-02-07,None


In [99]:
sum_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(alist))

In [40]:
ibesinfo3_sql="select * from IBESInfo3"
ibesinfo3=pd.read_sql(ibesinfo3_sql,cnxn)

In [186]:
ibesinfo3[ibesinfo3["Code"]==330]

,ITicker,Cusip,STicker,Name,Country,Currency_,OneTenth,SecType,ExchCtry,Exchange,Code
1209,AFL,00105510,AFL,AFLAC INC,NA,USD,0,S,NA,A,330
